## Core places processing
This notebook should read in the coreplaces files from the raw data folder and:
    1. Associate a date to each file
    2. Filter down each file to Philadelphia places
    3. Concatenate the files together
    4. Write out the result to a file in the proessed data folder

In [1]:
import pandas as pd
from safegraph_py_functions import safegraph_py_functions as sgpy
import os
from dotenv import load_dotenv, find_dotenv
import zipfile

In [2]:
# find .env automagically by walking up directories until it's found
dotenv_path = find_dotenv()

# load up the entries as environment variables
load_dotenv(dotenv_path)
root_dir = os.environ.get("ROOT_DIR")
raw_data_dir = os.path.join(root_dir,'data/raw')

In [3]:
# local directory where we want to put all the data
core_path = os.path.join(raw_data_dir,'core')
# print(local)
archives = []
# r=root, d=directories, f = files
for r, d, f in os.walk(core_path):
    for file in f:
        if file.endswith('.zip'):
            archives.append(os.path.join(core_path, r, file))
archives

['/Users/hannahkronenberg/SafegraphCOVIDPhilly/data/raw/core/2020/03/CoreRecords-CORE_POI-2019_03-2020-03-25.zip',
 '/Users/hannahkronenberg/SafegraphCOVIDPhilly/data/raw/core/2020/04/CoreApr2020Release-CORE_POI-2020_03-2020-04-07.zip',
 '/Users/hannahkronenberg/SafegraphCOVIDPhilly/data/raw/core/2020/05/CorePlacesMay2020Release-CORE_POI-2020_04-2020-05-06.zip',
 '/Users/hannahkronenberg/SafegraphCOVIDPhilly/data/raw/core/2020/10/Core-USA-Oct-CORE_POI-2020_09-2020-10-19.zip',
 '/Users/hannahkronenberg/SafegraphCOVIDPhilly/data/raw/core/2020/07/Core-USA-July2020-Release-CORE_POI-2020_06-2020-07-13.zip',
 '/Users/hannahkronenberg/SafegraphCOVIDPhilly/data/raw/core/2020/09/Core-USA-Sep-CORE_POI-2020_08-2020-09-08.zip',
 '/Users/hannahkronenberg/SafegraphCOVIDPhilly/data/raw/core/2020/08/Core-USA-August2020-Release-CORE_POI-2020_07-2020-08-07.zip',
 '/Users/hannahkronenberg/SafegraphCOVIDPhilly/data/raw/core/2020/06/Core-USA-June2020-Release-CORE_POI-2020_05-2020-06-06.zip']

In [4]:
for archive in archives:
    with zipfile.ZipFile(archive,"r") as zip_ref:
        zip_ref.extractall(archive[:-4])

In [5]:
files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(core_path):
    for file in f:
        if file.endswith('.csv.gz') and 'core_poi-part' in file:
            files.append(os.path.join(core_path, r, file))

In [6]:
files

['/Users/hannahkronenberg/SafegraphCOVIDPhilly/data/raw/core/2020/03/CoreRecords-CORE_POI-2019_03-2020-03-25/core_poi-part5.csv.gz',
 '/Users/hannahkronenberg/SafegraphCOVIDPhilly/data/raw/core/2020/03/CoreRecords-CORE_POI-2019_03-2020-03-25/core_poi-part3.csv.gz',
 '/Users/hannahkronenberg/SafegraphCOVIDPhilly/data/raw/core/2020/03/CoreRecords-CORE_POI-2019_03-2020-03-25/core_poi-part1.csv.gz',
 '/Users/hannahkronenberg/SafegraphCOVIDPhilly/data/raw/core/2020/03/CoreRecords-CORE_POI-2019_03-2020-03-25/core_poi-part4.csv.gz',
 '/Users/hannahkronenberg/SafegraphCOVIDPhilly/data/raw/core/2020/03/CoreRecords-CORE_POI-2019_03-2020-03-25/core_poi-part2.csv.gz',
 '/Users/hannahkronenberg/SafegraphCOVIDPhilly/data/raw/core/2020/04/CoreApr2020Release-CORE_POI-2020_03-2020-04-07/core_poi-part5.csv.gz',
 '/Users/hannahkronenberg/SafegraphCOVIDPhilly/data/raw/core/2020/04/CoreApr2020Release-CORE_POI-2020_03-2020-04-07/core_poi-part3.csv.gz',
 '/Users/hannahkronenberg/SafegraphCOVIDPhilly/data/raw

In [7]:
def extract_date(file):
    folders = file.split("/")
    date = folders[-2]
    date = date.split("-")[-3:]
    return '-'.join(date)
    

In [8]:
all_dates = list(set([extract_date(file) for file in files]))
all_dates.sort()
all_dates = ['1900-01-01'] + all_dates

In [9]:
# make mapping from file date to previous file date
date_df = pd.DataFrame({'valid_to':all_dates})
date_df['valid_from'] = date_df.shift()['valid_to']
date_df = date_df.set_index('valid_to')

In [10]:
date_df

,valid_from
valid_to,
1900-01-01,NaN
2020-03-25,1900-01-01
2020-04-07,2020-03-25
2020-05-06,2020-04-07
2020-06-06,2020-05-06
2020-07-13,2020-06-06
2020-08-07,2020-07-13
2020-09-08,2020-08-07
2020-10-19,2020-09-08


In [12]:
date_df.loc['2020-03-25', 'valid_from']

'1900-01-01'

In [11]:
keep_cols = ['safegraph_place_id', 'parent_safegraph_place_id', 'safegraph_brand_ids', 'brands', 'top_category', 
             'sub_category', 'naics_code', 'latitude', 'longitude', 'street_address', 'city', 'region', 
             'postal_code', 'open_hours']

def filter_and_format(file):
    df = pd.read_csv(file)
    df = df[keep_cols]
    # Philadelphia selection
    df['postal_code'] = df['postal_code'].apply(lambda x: ('00000'+str(x))[-5:])
    in_philly = df['postal_code'].astype(str).str.startswith("191")
    df = df.loc[in_philly]
    # The visits_by_day column contains a list of integers. 
    # This explodes that list so we get one row per day.
    file_date = extract_date(file)
    df['valid_to'] = file_date
    df['valid_to'] = pd.to_datetime(df['valid_to'], format = '%Y-%m-%d')
    df['valid_from'] = date_df.loc[file_date,'valid_from']
    df['valid_from'] = pd.to_datetime(df['valid_from'], format = '%Y-%m-%d')
    return df

In [12]:
example_df = filter_and_format(files[0])

In [13]:
example_df.head()

,safegraph_place_id,parent_safegraph_place_id,safegraph_brand_ids,brands,top_category,sub_category,naics_code,latitude,longitude,street_address,city,region,postal_code,open_hours,valid_to,valid_from
273,sg:13ff95596a514499a0edbc3d45e7ad98,NaN,NaN,NaN,Religious Organizations,Religious Organizations,813110.0,40.053464,-75.076514,7101 Pennway St,Philadelphia,PA,19111,"{ ""Mon"": [], ""Tue"": [], ""Wed"": [], ""Thu"": [], ...",2020-03-25,1900-01-01
599,sg:2b4441655d4a4e23ae20d327e67e219c,NaN,NaN,NaN,NaN,NaN,NaN,39.937086,-75.162869,1122 Washington Ave,Philadelphia,PA,19147,NaN,2020-03-25,1900-01-01
747,sg:37f738a1643f46dd95fcca5935fbe449,NaN,NaN,NaN,Elementary and Secondary Schools,Elementary and Secondary Schools,611110.0,39.983266,-75.246632,1141 N 63rd St,Philadelphia,PA,19151,NaN,2020-03-25,1900-01-01
892,sg:423a8dc1c6084d579729f06762ea748a,NaN,NaN,NaN,Motion Picture and Video Industries,Motion Picture Theaters (except Drive-Ins),512131.0,39.961404,-75.155315,1026 Spring Garden St,Philadelphia,PA,19123,NaN,2020-03-25,1900-01-01
1033,sg:4c7fb593fdc44f3c939a4741aa7a0c0e,NaN,NaN,NaN,Religious Organizations,Religious Organizations,813110.0,39.927161,-75.192737,28th & Snyder Ave,Philadelphia,PA,19102,NaN,2020-03-25,1900-01-01


In [14]:
example_df.postal_code.unique()

array(['19111', '19147', '19151', '19123', '19102', '19134', '19131',
       '19139', '19148', '19114', '19149', '19152', '19141', '19116',
       '19107', '19153', '19104', '19144', '19121', '19129', '19145',
       '19103', '19124', '19143', '19115', '19106', '19120', '19119',
       '19135', '19109', '19132', '19125', '19146', '19133', '19130',
       '19138', '19142', '19140', '19118', '19136', '19150', '19137',
       '19154', '19128', '19127', '19122', '19126', '19110', '19112',
       '19176', '19113', '19155', '19192'], dtype=object)

In [15]:
philly_places = [filter_and_format(file) for file in files]

In [16]:
philly_places = pd.concat(philly_places)

In [17]:
philly_places

,safegraph_place_id,parent_safegraph_place_id,safegraph_brand_ids,brands,top_category,sub_category,naics_code,latitude,longitude,street_address,city,region,postal_code,open_hours,valid_to,valid_from
273,sg:13ff95596a514499a0edbc3d45e7ad98,NaN,NaN,NaN,Religious Organizations,Religious Organizations,813110.0,40.053464,-75.076514,7101 Pennway St,Philadelphia,PA,19111,"{ ""Mon"": [], ""Tue"": [], ""Wed"": [], ""Thu"": [], ...",2020-03-25,1900-01-01
599,sg:2b4441655d4a4e23ae20d327e67e219c,NaN,NaN,NaN,NaN,NaN,NaN,39.937086,-75.162869,1122 Washington Ave,Philadelphia,PA,19147,NaN,2020-03-25,1900-01-01
747,sg:37f738a1643f46dd95fcca5935fbe449,NaN,NaN,NaN,Elementary and Secondary Schools,Elementary and Secondary Schools,611110.0,39.983266,-75.246632,1141 N 63rd St,Philadelphia,PA,19151,NaN,2020-03-25,1900-01-01
892,sg:423a8dc1c6084d579729f06762ea748a,NaN,NaN,NaN,Motion Picture and Video Industries,Motion Picture Theaters (except Drive-Ins),512131.0,39.961404,-75.155315,1026 Spring Garden St,Philadelphia,PA,19123,NaN,2020-03-25,1900-01-01
1033,sg:4c7fb593fdc44f3c939a4741aa7a0c0e,NaN,NaN,NaN,Religious Organizations,Religious Organizations,813110.0,39.927161,-75.192737,28th & Snyder Ave,Philadelphia,PA,19102,NaN,2020-03-25,1900-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1074700,sg:cc420ccc9f3e4636b3b24e5bf76e2730,NaN,NaN,NaN,Traveler Accommodation,Hotels (except Casino Hotels) and Motels,721110.0,40.103577,-75.005351,11580 Roosevelt Blvd,Philadelphia,PA,19116,NaN,2020-06-06,2020-05-06
1074763,sg:d190a8b5f5a2483987db81dcd88fa867,NaN,NaN,NaN,Restaurants and Other Eating Places,Full-Service Restaurants,722511.0,40.077121,-75.208852,8634 Germantown Ave,Philadelphia,PA,19118,"{ ""Mon"": [[""11:00"", ""23:00""]], ""Tue"": [[""11:00...",2020-06-06,2020-05-06
1074801,sg:d59e27fc19264a2e8c855407505ba370,NaN,NaN,NaN,Restaurants and Other Eating Places,Full-Service Restaurants,722511.0,40.022127,-75.077535,1613 Pratt St,Philadelphia,PA,19124,NaN,2020-06-06,2020-05-06
1075246,sg:f9b54b6cfbc44ddf8d74a501189abbea,sg:d8e253d8737547119e6d436bcf94a690,NaN,NaN,Restaurants and Other Eating Places,Snack and Nonalcoholic Beverage Bars,722515.0,39.954692,-75.200246,3900 Sansom St,Philadelphia,PA,19104,"{ ""Mon"": [[""11:00"", ""24:00""]], ""Tue"": [[""11:00...",2020-06-06,2020-05-06


In [18]:
processed_data_dir = os.path.join(root_dir,'data/processed')

In [19]:
philly_places.to_csv(os.path.join(processed_data_dir,'philly_places.csv.tar.gz'), index = False)

In [20]:
for file in files:
    os.unlink(file)